In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
import random
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Input,InputLayer, Activation, BatchNormalization
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D, SpatialDropout2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import random
import cv2
import os
import numpy as np
from keras.applications.resnet50 import ResNet50
# import efficientnet.keras as efn
from keras.models import Model
import keras
from keras.callbacks import EarlyStopping
img_size = 224

ImportError: ignored

In [ ]:
labels = ['cat', 'dog', 'wild']
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, img))
                resized_arr = cv2.resize(img, (img_size, img_size))
                gray=cv2.cvtColor(resized_arr, cv2.COLOR_BGR2GRAY)
                data.append([gray, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
train=get_data("../input/animal-faces/afhq/train")
test=get_data("../input/animal-faces/afhq/val")

In [ ]:
Xt = []
Yt = []#train
Xv = []
Yv = []#test

for feature, label in train:
  Xt.append(feature)
  Yt.append(label)

for feature, label in test:
  Xv.append(feature)
  Yv.append(label)

In [ ]:
# Normalize the data
Xt = np.array(Xt,dtype=np.float16) / 255
Xv = np.array(Xt,dtype=np.float16) / 255

In [ ]:
print(Xt.shape)
print(Xv.shape)

In [ ]:
Xt=Xt.reshape(-1, img_size, img_size, 1)
Yt = np.array(Yt)

Xv=Xv.reshape(-1, img_size, img_size, 1)
Yv = np.array(Yv)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(img_size, img_size, 1)))

model.add(ZeroPadding2D((3, 3)))
model.add(Conv2D(112, activation='relu', kernel_size=(3, 3)))
model.add(MaxPooling2D((2, 2), strides=(3, 3)))
model.add(Conv2D(72, activation='relu', kernel_size=(2, 2)))
model.add(MaxPooling2D((2, 2), strides=(3, 3)))
model.add(Conv2D(64, activation='relu', kernel_size=(2, 2)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Conv2D(32, activation='relu', kernel_size=(2, 2)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=(3, 3)))
model.add(Conv2D(16, activation='relu', kernel_size=(2, 2)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dense(3, activation="softmax"))
model.summary()

In [ ]:
rms= RMSprop(lr=0.00001)
model.compile(optimizer =rms , loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

In [ ]:
Xt,Xv,Yt,Yv = train_test_split(Xt,Yt,train_size = 0.8,test_size = 0.2,random_state =42)

In [ ]:
history = model.fit(Xt,Yt,epochs = 100,validation_data = (Xv,Yv))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict_classes(Xt)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(Yt, predictions, target_names = ['cat (Class 0)','dog (Class 1)','wild (Class 2)']))

In [ ]:
import sklearn.metrics as metrics
cm = metrics.confusion_matrix(YT, predictions)
print(cm)

NameError: ignored